In [ ]:
# Importación librerias y creación de dataframes

import pandas as pd 
import numpy as np
import json
import os
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score, confusion_matrix

df_test= pd.read_csv("../files/input/test_data.csv.zip",index_col=False, compression="zip")
df_train = pd.read_csv("../files/input/train_data.csv.zip",index_col=False, compression="zip")

In [ ]:
# Paso 1.
# Realice la limpieza de los datasets:
# - Renombre la columna "default payment next month" a "default".
# - Remueva la columna "ID".
# - Elimine los registros con informacion no disponible.
# - Para la columna EDUCATION, valores > 4 indican niveles superiores
#   de educación, agrupe estos valores en la categoría "others".
# - Renombre la columna "default payment next month" a "default"
# - Remueva la columna "ID".

df_train.rename(columns={"default payment next month": "default"}, inplace=True)
df_test.rename(columns={"default payment next month": "default"}, inplace=True)
df_train.drop(columns=["ID"], inplace=True)
df_test.drop(columns=["ID"], inplace=True)
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

df_train["EDUCATION"] = df_train["EDUCATION"].apply(lambda x: 4 if x>4 else x)
df_train['EDUCATION'] = df_train['EDUCATION'].apply(lambda x: x if x > 0 else np.nan)
df_train['MARRIAGE'] = df_train['MARRIAGE'].apply(lambda x: x if x > 0 else np.nan)

df_test["EDUCATION"] = df_test["EDUCATION"].apply(lambda x: 4 if x>4 else x)
df_test['EDUCATION'] = df_test['EDUCATION'].apply(lambda x: x if x > 0 else np.nan)
df_test['MARRIAGE'] = df_test['MARRIAGE'].apply(lambda x: x if x > 0 else np.nan)

In [ ]:
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.

x_train=df_train.drop(columns="default")
y_train=df_train["default"]


x_test=df_test.drop(columns="default")
y_test=df_test["default"]

In [ ]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Ajusta un modelo de bosques aleatorios (rando forest).

# Variables categoricas
categorical=["SEX","EDUCATION","MARRIAGE"]


preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(), categorical)], remainder="passthrough"
)


pipeline=Pipeline(
    [("preprocessor",preprocessor), ('classifier', RandomForestClassifier(random_state=42))]
)

In [ ]:
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use la función de precision
# balanceada para medir la precisión del modelo.

parametros = {
    'classifier__n_estimators': [200],
    'classifier__max_depth': [None],
    'classifier__min_samples_split': [10],
    'classifier__min_samples_leaf': [1,2],
    'classifier__max_features': ['sqrt']
}
modelo = GridSearchCV(pipeline, parametros, cv=10, scoring='balanced_accuracy', n_jobs=-1, refit=True)
modelo.fit(x_train, y_train)

In [ ]:
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

import pickle
import gzip
import os

models_direccion = '../files/models'
os.makedirs(models_direccion, exist_ok=True)

with gzip.open(os.path.join(models_direccion, 'model.pkl.gz'), 'wb') as file:
    pickle.dump(modelo, file)

In [ ]:
# Paso 6.
# Calcule las metricas de precision, precision balanceada, recall,
# y f1-score para los conjuntos de entrenamiento y prueba.
# Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# Este diccionario tiene un campo para indicar si es el conjunto
# de entrenamiento o prueba. Por ejemplo:
#
# {'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
# {'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}

y_train_pred = modelo.predict(x_train)
y_test_pred = modelo.predict(x_test)

metricas_train = {
    'type': 'metrics',
    'dataset': 'train',
    'precision': precision_score(y_train, y_train_pred),
    'balanced_accuracy': balanced_accuracy_score(y_train, y_train_pred),
    'recall': recall_score(y_train, y_train_pred),
    'f1_score': f1_score(y_train, y_train_pred)
}

metricas_test = {
    'type': 'metrics',
    'dataset': 'test',
    'precision': precision_score(y_test, y_test_pred),
    'balanced_accuracy': balanced_accuracy_score(y_test, y_test_pred),
    'recall': recall_score(y_test, y_test_pred),
    'f1_score': f1_score(y_test, y_test_pred)
}

os.makedirs('../files/output', exist_ok=True)

metricas_direccion = '../files/output/metrics.json'

with open(metricas_direccion, 'w') as f:
    json.dump([metricas_train], f)
    f.write('\n')
    json.dump([metricas_test], f)
    f.write('\n')

In [ ]:
# Paso 7.
# Calcule las matrices de confusion para los conjuntos de entrenamiento y
# prueba. Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
# {'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}
#

cm_train = confusion_matrix(y_train, y_train_pred)
cm_test = confusion_matrix(y_test, y_test_pred)

cm_matrix_train = {
    'type': 'cm_matrix',
    'dataset': 'train',
    'true_0': {"predicted_0": cm_train[0, 0], "predicted_1": cm_train[0, 1]},
    'true_1': {"predicted_0": cm_train[1, 0], "predicted_1": cm_train[1, 1]}
}

cm_matrix_test = {
    'type': 'cm_matrix',
    'dataset': 'test',
    'true_0': {"predicted_0": cm_test[0, 0], "predicted_1": cm_test[0, 1]},
    'true_1': {"predicted_0": cm_test[1, 0], "predicted_1": cm_test[1, 1]}
}

def convert_to_serializable(obj):
    if isinstance(obj, np.int64):
        return int(obj)
    raise TypeError(f"Type {type(obj)} not serializable")

# Usar la función con json.dump
with open(metricas_direccion, 'w') as f:
    json.dump(metricas_train, f, default=convert_to_serializable)
    f.write('\n')
    json.dump(metricas_test, f, default=convert_to_serializable)
    f.write('\n')
    json.dump(cm_matrix_train, f, separators=(',', ':'), ensure_ascii=False, default=convert_to_serializable)
    f.write("\n")
    json.dump(cm_matrix_test, f, separators=(',', ':'), ensure_ascii=False, default=convert_to_serializable)